In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
from skimage.measure import block_reduce

In [2]:
images = [f for f in os.listdir()]
images.sort()
jpgs = [f for f in images if f.endswith(".jpg")]
pngs = [f for f in images if f.endswith(".png")]

In [3]:
IMAGE_RESOLUTION = (640, 480)
LABEL_RESOLUTION = (20, 15)
BLOCK_SIZE=IMAGE_RESOLUTION[0] // LABEL_RESOLUTION[0]

In [4]:
X = np.zeros((len(jpgs), IMAGE_RESOLUTION[1], IMAGE_RESOLUTION[0], 3), dtype=np.float32)
y = np.zeros((len(jpgs), LABEL_RESOLUTION[1], LABEL_RESOLUTION[0], 3), dtype=np.float32)

In [5]:
for idx, (jpg, png) in enumerate(zip(jpgs, pngs)):
    image = cv2.imread(jpg)[:,:,::-1]
    label = cv2.imread(png)[:,:,::-1]
    # portrait/landscape maybe different in image and label
    if image.shape[0] != label.shape[0]:
        image = image.swapaxes(0,1)[:,::-1,:]
    # flip portrait images to landscape
    if image.shape[0] > image.shape[1]:
        image = image.swapaxes(0,1)[:,::-1,:]
        label = label.swapaxes(0,1)[:,::-1,:]
    image = cv2.resize(image, IMAGE_RESOLUTION) / 255.0
    label = cv2.resize(label, IMAGE_RESOLUTION) / 255.0
    label = block_reduce(label, (BLOCK_SIZE,BLOCK_SIZE,1), func=np.mean)
    X[idx,:,:,:] = image
    y[idx,:,:,:] = label

In [6]:
# binarize favoring non-background classes
y[:,:,:,0] = (y[:,:,:,0]==1).astype(np.float32)
y[:,:,:,1] = np.ceil(y[:,:,:,1])
y[:,:,:,2] = np.ceil(y[:,:,:,2])

In [ ]:
np.savez("X.npz", X)
np.savez("y.npz", y)